# SIR Model without vital dynamics

In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt

import warnings

from ipywidgets import interact, HBox, Label
import ipywidgets as widgets

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, HoverTool
output_notebook()

Loading BokehJS ...

### Model

In [ ]:
def sir(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt   

In [ ]:
#Initial conditions

N = 1000 #population
I0, R0 = 1, 0 #infected & removed
S0 = N - I0 - R0 #susceptible
beta, gamma = 0.2, 1./10 #contact rate & recovery rate

t = np.linspace(0, 50, 50)
y0 = S0, I0, R0

y = odeint(sir, y0, t, args=(N, beta, gamma))
S, I, R = y.T

x = t
y1 = S
y2 = I
y3 = R

## Plot and simulate the model - Trend over time

In [ ]:
p = figure(plot_width=700, plot_height=400, title="SIR model without vital dynamics", 
           tools=["pan,wheel_zoom,box_zoom,reset,tap"], x_axis_label = 'Days', y_axis_label = 'Count')

hover = HoverTool(
        tooltips=[
            ("Count", "@y"),
            ("Day", "@x"),
        ]
    )
p.add_tools(hover)
source1 = ColumnDataSource(data=dict(x=x, y=y1))
source2 = ColumnDataSource(data=dict(x=x, y=y2))
source3 = ColumnDataSource(data=dict(x=x, y=y3))

s = p.line('x', 'y', source=source1, color="darkcyan", line_width=1.5, alpha=0.8, legend='Susceptible')
i = p.line('x', 'y', source=source2, color="red", line_width=1.5, alpha=0.8, legend='Infected')
r = p.line('x', 'y', source=source3, color="grey", line_width=1.5, alpha=0.8, legend='Recovered')

p.x_range.range_padding = 0
p.y_range.range_padding = 0
p.min_border = 0
p.legend.location='top_left'
p.legend.reverse()

In [ ]:
def update(pop=1000, init_infected=1, init_recovered=0, infect_rate=0.2, recover_rate=1/10,time_span=100):
    n,i0,r0,beta,gamma,T = pop, init_infected, init_recovered, infect_rate, recover_rate, time_span
    y0 = n-i0-r0, i0, r0
    t = np.linspace(0,T,T)
    y = odeint(sir, y0, t, args=(n, beta, gamma))
    S, I, R = y.T
    s.data_source.data['x'] = t
    i.data_source.data['x'] = t    
    r.data_source.data['x'] = t
    s.data_source.data['y'] = S
    i.data_source.data['y'] = I
    r.data_source.data['y'] = R
    push_notebook()

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
interact(update,pop=(100,10000,100), init_infected=(0,100,1), init_recovered=(0,100,1), infect_rate=(0,1,0.01), recover_rate=(0,1,0.01), time_span=(0,500))
show(p, notebook_handle=True)

interactive(children=(IntSlider(value=1000, description='pop', max=10000, min=100, step=100), IntSlider(value=…

## Plot and simulate the model - Composition

In [ ]:
df = pd.DataFrame()
df['S'] = y1
df['I'] = y2
df['R'] = y3
v = 3

In [ ]:
m = figure(plot_width=700, plot_height=400, title="SIR without vital dynamics", 
           tools=["pan,wheel_zoom,box_zoom,reset,tap"], x_axis_label = 'Days', y_axis_label = 'Count')
names = ['I','S','R']
stack = m.varea_stack(['I','S','R'], x='index', color=['red','darkcyan','grey'], alpha=0.5, source=df, legend=['Infected','Susceptible','Recovered'])
m.x_range.range_padding = 0
m.y_range.range_padding = 0
m.min_border = 0
m.legend.location='top_left'
m.legend.reverse()

In [ ]:
def update_area(pop=1000, init_infected=1, init_recovered=0, infect_rate=0.2, recover_rate=1/10,time_span=100):
    n,i0,r0,beta,gamma,T = pop, init_infected, init_recovered, infect_rate, recover_rate, time_span
    y0 = n-i0-r0, i0, r0
    t = np.linspace(0,T,T)
    y = odeint(sir, y0, t, args=(n, beta, gamma))
    S, I, R = y.T
    df = pd.DataFrame()
    df['S'] = S
    df['I'] = I
    df['R'] = R
    stack[0].data_source.data['I'] = df['I']
    stack[0].data_source.data['S'] = df['S']
    stack[0].data_source.data['R'] = df['R']
    stack[0].data_source.data['index'] = df.index.values
    stack[0].data_source.data['source'] = df
    stack[1].data_source.data['I'] = df['I']
    stack[1].data_source.data['S'] = df['S']
    stack[1].data_source.data['R'] = df['R']
    stack[1].data_source.data['index'] = df.index.values
    stack[2].data_source.data['I'] = df['I']
    stack[2].data_source.data['S'] = df['S']
    stack[2].data_source.data['R'] = df['R']
    stack[2].data_source.data['index'] = df.index.values
    push_notebook()

In [ ]:
interact(update_area,pop=(100,10000,100), init_infected=(0,100,1), init_recovered=(0,100,1), infect_rate=(0,1,0.01), recover_rate=(0,1,0.01), time_span=(0,500))
show(m, notebook_handle=True)

interactive(children=(IntSlider(value=1000, description='pop', max=10000, min=100, step=100), IntSlider(value=…